<a href="https://colab.research.google.com/github/03axdov/Notebooks/blob/main/SentimentClassficiationII.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

import tensorflow_datasets as tfds
import tensorflow as tf

tfds.disable_progress_bar()

In [2]:
import matplotlib.pyplot as plt

def plot_graphs(history, metric):
  plt.plot(history.history[metric], label=metric)
  plt.plot(history.history["val_" + metric], label="val_"+metric)
  plt.ylabel(metric)
  plt.xlabel("Epochs")
  plt.legend(loc="upper left")

In [3]:
dataset, info = tfds.load(
    "imdb_reviews", with_info=True,
    as_supervised=True,
)

train_dataset, test_dataset = dataset["train"], dataset["test"]

Dataset imdb_reviews downloaded and prepared to ~/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [4]:
i = 0
dataset = iter(train_dataset)
while i < 3:
  text, label = next(dataset)
  print(str(text.numpy().decode()))
  print(label.numpy())
  i += 1

This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.
0
I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell asleep because the film was rubbish. The plot development was co

In [5]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64

In [6]:
train_dataset = train_dataset.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [7]:
for example, label in train_dataset.take(1):
  print('texts: ', example.numpy()[:3])
  print()
  print('labels: ', label.numpy()[:3])

texts:  [b'I\'m hearing rumors of an upcoming "Leonard Nimoy Demonstrates the Blu-ray Disc". With advances over the past 25 years ranging from Steady-cam to CGI, it\'ll be interesting to see if the franchise can be reinvigorated. I just hope it helps to remove the bad taste left in my mouth by that whole Magnavision demonstration fiasco.<br /><br />And yes... "Leonard Nimoy Demonstrates the Betamax VCR" was a brilliant milestone in entertainment history. After the tentative "Leonard Nimoy Demonstrates the Compact Cassette" and the downright tacky "Leonard Nimoy Demonstrates the 8-Track Tape", who would have expected such a glorious piece of cinema? I\'m weeping right now just thinking about it.'
 b'I remember my parents not understanding Saturday Night Live when I was 15. They also did not understand Rock n Roll and many other things. Now that I am approaching their age, I still remember, and find I understand many of the things my kids love. But this is pathetic. I cannot say I have s

In [8]:
VOCAB_SIZE = 1000
encoder = tf.keras.layers.TextVectorization(
    max_tokens=VOCAB_SIZE
)
encoder.adapt(train_dataset.map(lambda x, y: x))

In [9]:
vocab = np.array(encoder.get_vocabulary())
vocab[:20]

array(['', '[UNK]', 'the', 'and', 'a', 'of', 'to', 'is', 'in', 'it', 'i',
       'this', 'that', 'br', 'was', 'as', 'for', 'with', 'movie', 'but'],
      dtype='<U14')

In [10]:
encoded_example = encoder(example)[:3].numpy()
encoded_example

array([[142,   1,   1, ...,   0,   0,   0],
       [ 10, 364,  56, ...,   0,   0,   0],
       [ 11, 227,  28, ...,   0,   0,   0]])

In [11]:
for n in range(3):
  print(f"Original: {example[n].numpy()}")
  print(f"Round trip: {' '.join(vocab[encoded_example[n]])}")
  print("")

Original: b'I\'m hearing rumors of an upcoming "Leonard Nimoy Demonstrates the Blu-ray Disc". With advances over the past 25 years ranging from Steady-cam to CGI, it\'ll be interesting to see if the franchise can be reinvigorated. I just hope it helps to remove the bad taste left in my mouth by that whole Magnavision demonstration fiasco.<br /><br />And yes... "Leonard Nimoy Demonstrates the Betamax VCR" was a brilliant milestone in entertainment history. After the tentative "Leonard Nimoy Demonstrates the Compact Cassette" and the downright tacky "Leonard Nimoy Demonstrates the 8-Track Tape", who would have expected such a glorious piece of cinema? I\'m weeping right now just thinking about it.'
Round trip: im [UNK] [UNK] of an [UNK] [UNK] [UNK] [UNK] the [UNK] [UNK] with [UNK] over the past [UNK] years [UNK] from [UNK] to [UNK] [UNK] be interesting to see if the [UNK] can be [UNK] i just hope it [UNK] to [UNK] the bad [UNK] left in my [UNK] by that whole [UNK] [UNK] [UNK] br and yes 

In [13]:
model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=64,
        mask_zero=True
    ),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation="relu"),
    tf.keras.layers.Dense(1)
])

In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVec  (None, None)             0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, None, 64)          64000     
                                                                 
 bidirectional (Bidirectiona  (None, 128)              66048     
 l)                                                              
                                                                 
 dense (Dense)               (None, 64)                8256      
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 138,369
Trainable params: 138,369
Non-trai

In [15]:
print([layer.supports_masking for layer in model.layers])

[False, True, True, True, True]


In [16]:
sample_text = ("The movie was cool. Great acting and interesting plot lines. Would definitely recommend this movie!")

predictions = model.predict(np.array([sample_text]))
print(predictions[0])

[0.01335325]


In [17]:
padding = "the" * 2000
predictions = model.predict(np.array([sample_text, padding]))
print(predictions[0])

[0.01335325]


In [18]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=["accuracy"]
)

In [ ]:
EPOCHS = 10

history = model.fit(
    train_dataset,
    validation_data=test_dataset,
    epochs=EPOCHS,
    validation_steps=30
)